# Demand

In [1]:
import pandas as pd
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_palette("GnBu_d")

In [2]:
brm = pd.read_csv("../generated/brm_cleaned.csv")
frb = pd.read_csv("../generated/frb_cleaned.csv")

In [3]:
def calc_demand(frame):
    frame["trip_duration"] = frame["trip_duration"].apply(pd.Timedelta) #convert time column to a useful type
    daily_driven = frame.groupby("day").agg({"trip_duration":"sum"}) #aggregate daily driven time
    #For a more detailed aggregation we add a daytime to the variables, so we can calculate time driven by daytime.
    #This also allows a better mapping of how weather effects demand.
    morning = frame[(frame["time"] > "06:00:00") & (frame["time"] <= "11:00:00")] #saving rows where time is in defined range to a new df
    noon = frame[(frame["time"] > "11:00:00") & (frame["time"] <= "16:00:00")]
    evening = frame[(frame["time"] > "16:00:00") & (frame["time"] <= "21:00:00")]
    night = frame[(frame["time"] > "21:00:00")] + frame[(frame["time"] <= "06:00:00")]
    for i in morning.index: # using indexes from lists above to cycle through the whole data
       frame.at[i , "daytime"] = 0 # morning refers to 0
    for i in noon.index:
       frame.at[i , "daytime"] = 1 # noon -> 1
    for i in evening.index:
       frame.at[i , "daytime"] = 2 # evening -> 2
    for i in night.index:
       frame.at[i , "daytime"] = 3 # night -> 3
    frame["daytime"] = frame["daytime"].map(lambda x: (int(x))) #map integer for daytime instead of float
    return frame
    

summarizing data over day and daytime via aggregation of the trip durations taken.

In [4]:
frb_demand = calc_demand(frb)
frb_daytime_driven = frb_demand.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
frb_daytime_driven["trip_duration"] = frb_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")

In [5]:
brm_demand = calc_demand(brm)
brm_daytime_driven = brm_demand.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
brm_daytime_driven["trip_duration"] = brm_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")

export data by hours and summarizing the trips taken in these time sections

In [6]:
frb_demand["hour"] = frb_demand["time"].str[:-6]
frb_hourly_driven = frb_demand.groupby(["day","hour"]).agg({"trip_duration":"sum"})
frb_hourly_driven.to_csv("../generated/frb_hourly_driven.csv")

In [7]:
brm["hour"] = brm["time"].str[:-6]
brm_hourly_driven = brm.groupby(["day","hour"]).agg({"trip_duration":"sum"})
brm_hourly_driven.to_csv("../generated/brm_hourly_driven.csv")